In [18]:
import os, httpx, json
from pprint import pprint

API_KEY = os.getenv("THESPORTSDB_KEY", "3")  # free demo key
BASE_URL = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}"
print("Using BASE_URL:", BASE_URL)

def GET(path, params=None):
    url = f"{BASE_URL}/{path.lstrip('/')}"
    r = httpx.get(url, params=params or {}, timeout=20)
    r.raise_for_status()
    data = r.json()
    # Normalize empty payloads like {"teams": None}
    if isinstance(data, dict) and data and all(v is None for v in data.values()):
        return {}
    return data

Using BASE_URL: https://www.thesportsdb.com/api/v1/json/3


In [19]:
sports = GET("all_sports.php").get("sports") or []
print("Total sports:", len(sports))
pprint(sports[0])  # first full record

Total sports: 1
{'idSport': '102',
 'strFormat': 'TeamvsTeam',
 'strSport': 'Soccer',
 'strSportDescription': 'Association football, more commonly known as football '
                        'or soccer, is a team sport played between two teams '
                        'of eleven players with a spherical ball. It is played '
                        'by 250 million players in over 200 countries and '
                        "dependencies, making it the world's most popular "
                        'sport. The game is played on a rectangular field with '
                        'a goal at each end. The object of the game is to '
                        'score by getting the ball into the opposing goal.\r\n'
                        '\r\n'
                        'Players are not allowed to touch the ball with their '
                        'hands or arms while it is in play, unless they are '
                        'goalkeepers (and then only when within their penalty '
               

In [20]:
leagues = GET("all_leagues.php").get("leagues") or []
print("Total leagues:", len(leagues))
pprint(leagues[2])  # see all keys of one league

Total leagues: 50
{'idLeague': '4330',
 'strLeague': 'Scottish Premier League',
 'strLeagueAlternate': 'Scottish Premiership, SPFL',
 'strSport': 'Soccer'}


In [ ]:
LEAGUE_ID = "4330"  
teams = GET("lookup_all_teams.php", {"id": LEAGUE_ID}).get("teams") or []
print("Teams in league", LEAGUE_ID, ":", len(teams))
pprint(teams[0])  # one team record with all fields

Teams in league 4330 : 24
{'idAPIfootball': '68',
 'idESPN': '358',
 'idLeague': '4396',
 'idLeague2': '4482',
 'idLeague3': '4570',
 'idLeague4': '4847',
 'idLeague5': None,
 'idLeague6': None,
 'idLeague7': None,
 'idTeam': '133606',
 'idVenue': '28826',
 'intFormedYear': '1874',
 'intLoved': None,
 'intStadiumCapacity': '28723',
 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/yvxxrv1448808301.png',
 'strBanner': 'https://r2.thesportsdb.com/images/media/team/banner/k17h8h1528107822.jpg',
 'strColour1': '',
 'strColour2': '',
 'strColour3': '',
 'strCountry': 'England',
 'strDescriptionCN': None,
 'strDescriptionDE': None,
 'strDescriptionEN': 'Bolton Wanderers Football Club is a professional '
                     'football club based in Horwich, Bolton, England, which '
                     'competes in League Two, the fourth tier of English '
                     'football.\r\n'
                     '\r\n'
                     'Formed as Christ Church Football Club

In [23]:
seasons = GET("search_all_seasons.php", {"id": LEAGUE_ID}).get("seasons") or []
print("Seasons for league", LEAGUE_ID, ":", len(seasons))
pprint(seasons)

Seasons for league 4330 : 14
[{'strSeason': '2012-2013'},
 {'strSeason': '2013-2014'},
 {'strSeason': '2014-2015'},
 {'strSeason': '2015-2016'},
 {'strSeason': '2016-2017'},
 {'strSeason': '2017-2018'},
 {'strSeason': '2018-2019'},
 {'strSeason': '2019-2020'},
 {'strSeason': '2020-2021'},
 {'strSeason': '2021-2022'},
 {'strSeason': '2022-2023'},
 {'strSeason': '2023-2024'},
 {'strSeason': '2024-2025'},
 {'strSeason': '2025-2026'}]


In [ ]:
past_matches = GET("eventspastleague.php", {"id": LEAGUE_ID}).get("events") or []
next_matches = GET("eventsnextleague.php", {"id": LEAGUE_ID}).get("events") or []
print("Past matches:", len(past_matches), "| Next matches:", len(next_matches))
pprint(past_matches[0])  # see all fields for one match

In [ ]:
# Cell — Fetch teams in a league via the agent
import requests, json

AGENT_URL = "http://127.0.0.1:8000/collect"   # change if different
LEAGUE_NAME = "English Premier League"        # try others too

payload = {
    "intent": "teams.list",
    "args": { "leagueName": LEAGUE_NAME }
}

resp = requests.post(AGENT_URL, json=payload).json()

print(f"Teams for '{LEAGUE_NAME}':", len(resp.get("data", {}).get("teams") or []))
for t in resp.get("data", {}).get("teams", [])[:10]:   # show first 10
    print("-", t.get("idTeam"), "|", t.get("strTeam"))

# Show all available fields for the first team
if resp.get("data", {}).get("teams"):
    print("\nSample team record (first team):")
    team = resp["data"]["teams"][0]
    for k, v in list(team.items())[:20]:
        print(f"{k:20} {v}")

Teams for 'English Premier League': 0


: 

In [ ]:
last_team_matches = GET("eventslast.php", {"id": TEAM_ID}).get("results") or []
print("Last matches for team:", len(last_team_matches))
pprint(last_team_matches[0])

In [ ]:
EVENT_ID = past_matches[0]["idEvent"]  # take first past match
detail = GET("lookupevent.php", {"id": EVENT_ID}).get("events") or []
timeline = GET("lookuptimeline.php", {"id": EVENT_ID}).get("timeline") or []
stats = GET("lookupeventstats.php", {"id": EVENT_ID}).get("eventstats") or []
lineup = GET("lookuplineup.php", {"id": EVENT_ID}).get("lineup") or []

print("Event detail keys:", list(detail[0].keys()))
print("\nTimeline rows:", len(timeline))
pprint(timeline[:3])
print("\nStats rows:", len(stats))
pprint(stats[:3])
print("\nLineup rows:", len(lineup))
pprint(lineup[:1])

In [ ]:
VENUE_ID = "16163"  # Emirates Stadium
venue = GET("lookupvenue.php", {"id": VENUE_ID}).get("venues") or []
print("Venue fields:", list(venue[0].keys()))
pprint(venue[0])

In [ ]:
import httpx

API_KEY = "3"  # free demo key
BASE_URL = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}"

endpoints = [
    "all_sports.php",
    "all_leagues.php",
    "search_all_leagues.php",
    "lookupleague.php",
    "lookup_all_teams.php",
    "searchteams.php",
    "lookupteam.php",
    "lookup_all_players.php",
    "searchplayers.php",
    "lookupplayer.php",
    "eventspastleague.php",
    "eventsnextleague.php",
    "eventslast.php",
    "eventsnext.php",
    "eventsseason.php",
    "eventsday.php",
    "lookupevent.php",
    "lookuptimeline.php",
    "lookupeventstats.php",
    "lookuplineup.php",
    "lookupvenue.php",
]

for ep in endpoints:
    url = f"{BASE_URL}/{ep}"
    try:
        r = httpx.get(url, timeout=10)
        if r.status_code == 200:
            print(f"✅ {ep}")
        else:
            print(f"⚠️ {ep} -> {r.status_code}")
    except Exception as e:
        print(f"❌ {ep} -> {e}")

In [ ]:
# Cell 2 — choose a match (event)
# OPTION A: paste a known event id
EVENT_ID = "2267081"  # <- replace with any event id you want

# OPTION B: quickly grab one from a league's recent/next fixtures
# (Uncomment to auto-pick the first from Premier League)
# league_id = "4328"
# past = GET("eventspastleague.php", {"id": league_id}).get("events") or []
# nxt  = GET("eventsnextleague.php", {"id": league_id}).get("events") or []
# EVENT_ID = (past or nxt)[0]["idEvent"]
# print("Auto-picked EVENT_ID:", EVENT_ID)

EVENT_ID

In [ ]:
# Base config
BASE = "http://127.0.0.1:8000/collect"   # change if your server is elsewhere

import requests, json, time
from typing import Dict, Any, Optional, List

def call(intent: str, args: Dict[str, Any] | None = None) -> Dict[str, Any]:
    payload = {"intent": intent, "args": args or {}}
    r = requests.post(BASE, json=payload, timeout=30)
    try:
        data = r.json()
    except Exception as e:
        raise RuntimeError(f"Non-JSON response (status={r.status_code}): {r.text}") from e
    return data

def assert_ok(resp: Dict[str, Any], msg: str = ""):
    if not resp.get("ok", False):
        pretty = json.dumps(resp, indent=2)
        raise AssertionError(f"Expected ok=True. {msg}\nResponse:\n{pretty}")

def show(resp: Dict[str, Any], keys: Optional[List[str]] = None, maxlen: int = 3):
    """Pretty-print a small slice of the response for quick human inspection."""
    print(json.dumps({
        "ok": resp.get("ok"),
        "intent": resp.get("intent"),
        "args_resolved": resp.get("args_resolved"),
        "data_preview": {
            k: (resp["data"].get(k)[:maxlen] if isinstance(resp["data"].get(k), list) else resp["data"].get(k))
            for k in (keys or list((resp.get("data") or {}).keys()))
        }
    }, indent=2))

In [ ]:
# Fetch leagues
leagues_resp = call("leagues.list", {})
assert_ok(leagues_resp, "leagues.list failed")
show(leagues_resp, keys=["leagues"], maxlen=5)

# Pick English Premier League if present, else take the first
leagues = leagues_resp["data"].get("leagues") or []
assert leagues, "No leagues returned"

epl = next((l for l in leagues if (l.get("strLeague") or "").lower() == "english premier league"), leagues[0])
LEAGUE_ID = epl["idLeague"]
LEAGUE_NAME = epl["strLeague"]
print("Chosen League:", LEAGUE_NAME, LEAGUE_ID)

In [ ]:
seasons_resp = call("seasons.list", {"leagueId": LEAGUE_ID})
assert_ok(seasons_resp, "seasons.list failed")
show(seasons_resp, keys=["seasons"], maxlen=10)

seasons = seasons_resp["data"].get("seasons") or []
assert seasons, "No seasons returned for league"
SEASON = seasons[0]["strSeason"]
print("Chosen Season:", SEASON)

In [ ]:
teams_resp = call("teams.list", {"leagueId": LEAGUE_ID})
assert_ok(teams_resp, "teams.list failed")
show(teams_resp, keys=["teams"], maxlen=10)

teams = teams_resp["data"].get("teams") or []
assert teams, "No teams returned for league"
TEAM = teams[0]
TEAM_ID = TEAM["idTeam"]
TEAM_NAME = TEAM["strTeam"]
print("Chosen Team:", TEAM_NAME, TEAM_ID)

In [ ]:
team_detail_resp = call("team.get", {"teamId": TEAM_ID})
assert_ok(team_detail_resp, "team.get failed")
show(team_detail_resp, keys=["team"])

In [ ]:
players_resp = call("players.list", {"teamId": TEAM_ID})
assert_ok(players_resp, "players.list failed")
show(players_resp, keys=["players"], maxlen=10)

players = players_resp["data"].get("players") or []
if players:
    PLAYER_ID = players[0]["idPlayer"]
    PLAYER_NAME = players[0]["strPlayer"]
    print("Chosen Player:", PLAYER_NAME, PLAYER_ID)
else:
    PLAYER_ID = None
    PLAYER_NAME = None
    print("No players found for this team (can happen on some datasets).")

In [ ]:
if PLAYER_ID:
    player_detail_resp = call("player.get", {"playerId": PLAYER_ID})
    assert_ok(player_detail_resp, "player.get failed")
    show(player_detail_resp, keys=["player"])
else:
    print("Skipping player.get — no player chosen.")

In [ ]:
events_resp = call("events.list", {"leagueId": LEAGUE_ID, "season": SEASON})
assert_ok(events_resp, "events.list (league+season) failed")
show(events_resp, keys=["events"], maxlen=10)

events = events_resp["data"].get("events") or []
if not events:
    print("No events returned for this league+season (try a different season).")
EVENT = events[0] if events else None
EVENT_ID = EVENT["idEvent"] if EVENT else None
print("Chosen Event:", EVENT_ID)

In [ ]:
if EVENT_ID:
    event_detail_resp = call("event.get", {"eventId": EVENT_ID, "expand": ["timeline", "stats", "lineup"]})
    assert_ok(event_detail_resp, "event.get failed")
    show(event_detail_resp, keys=["event", "timeline", "stats", "lineup"], maxlen=5)
else:
    print("Skipping event.get — no event chosen.")

In [ ]:
from datetime import date
today_str = date.today().isoformat()

events_by_day_resp = call("events.list", {"date": today_str})
# Could be ok=false if API has no events for today; handle gracefully
if events_by_day_resp.get("ok"):
    show(events_by_day_resp, keys=["events"], maxlen=10)
    print("Events-by-day count:", len(events_by_day_resp["data"].get("events") or []))
else:
    print("No events for today or error:", json.dumps(events_by_day_resp, indent=2))

In [ ]:
# LAST events for team
last_resp = call("events.list", {"teamId": TEAM_ID, "kind": "last"})
assert_ok(last_resp, "events.list (team last) failed")
show(last_resp, keys=["events"], maxlen=5)

# NEXT events for team
next_resp = call("events.list", {"teamId": TEAM_ID, "kind": "next"})
assert_ok(next_resp, "events.list (team next) failed")
show(next_resp, keys=["events"], maxlen=5)

In [ ]:
def pass_fail(intent: str, args: Dict[str, Any] | None = None) -> bool:
    try:
        resp = call(intent, args or {})
        assert_ok(resp)
        return True
    except Exception as e:
        print(f"[FAIL] {intent} {args} -> {e}")
        return False

results = {
    "leagues.list": pass_fail("leagues.list"),
    "seasons.list": pass_fail("seasons.list", {"leagueId": LEAGUE_ID}),
    "teams.list":   pass_fail("teams.list", {"leagueId": LEAGUE_ID}),
    "team.get":     pass_fail("team.get", {"teamId": TEAM_ID}),
    "players.list": pass_fail("players.list", {"teamId": TEAM_ID}),
}

if PLAYER_ID:
    results["player.get"] = pass_fail("player.get", {"playerId": PLAYER_ID})

if EVENT_ID:
    results["events.list (league+season)"] = pass_fail("events.list", {"leagueId": LEAGUE_ID, "season": SEASON})
    results["event.get"] = pass_fail("event.get", {"eventId": EVENT_ID, "expand": ["timeline"]})

print("\nSMOKE SUMMARY")
for k, v in results.items():
    print(f"{'PASS' if v else 'FAIL'} - {k}")

In [ ]:
import requests, time, re, csv, json
from typing import Dict, Any, List, Optional

BASE = "http://127.0.0.1:8000/collect"  # your FastAPI agent

def call(intent: str, args: Dict[str, Any] | None = None, timeout=60) -> Dict[str, Any]:
    r = requests.post(BASE, json={"intent": intent, "args": args or {}}, timeout=timeout)
    try:
        data = r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON from agent (status {r.status_code}): {r.text[:500]}")
    return data

def assert_ok(resp: Dict[str, Any], ctx: str = "") -> Dict[str, Any]:
    if not resp.get("ok"):
        raise RuntimeError(f"Agent error {ctx}: {json.dumps(resp, indent=2)[:1000]}")
    return resp

# URL detectors
VIDEO_PAT = re.compile(r"(youtu\.be|youtube\.com|vimeo\.com|dai\.ly|dailymotion\.com|\.m3u8(\?|$)|\.mp4(\?|$)|\.mov(\?|$)|\.webm(\?|$))", re.I)

def pick_video_fields(obj: dict) -> List[str]:
    """Collect any string fields that look like video URLs."""
    out = []
    for k, v in (obj or {}).items():
        if isinstance(v, str) and "http" in v and VIDEO_PAT.search(v):
            out.append(v.strip())
    # de-dup, keep order
    seen, uniq = set(), []
    for u in out:
        if u not in seen:
            uniq.append(u); seen.add(u)
    return uniq

def write_csv(path: str, rows: List[dict], fieldnames: List[str]):
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in rows:
            w.writerow({k: r.get(k, "") for k in fieldnames})

print("Agent:", BASE)

In [ ]:
import requests, time, re, json
from typing import Dict, Any, List, Optional

BASE = "http://127.0.0.1:8000/collect"  # your FastAPI agent

VIDEO_PAT = re.compile(r"(youtu\.be|youtube\.com|vimeo\.com|dai\.ly|dailymotion\.com|\.m3u8(\?|$)|\.mp4(\?|$)|\.mov(\?|$)|\.webm(\?|$))", re.I)

def pick_video_fields(obj: dict) -> List[str]:
    out = []
    for k, v in (obj or {}).items():
        if isinstance(v, str) and "http" in v and VIDEO_PAT.search(v):
            out.append(v.strip())
    # de-dup preserving order
    seen, uniq = set(), []
    for u in out:
        if u not in seen:
            uniq.append(u); seen.add(u)
    return uniq

def agent_call(intent: str, args: Dict[str, Any] | None = None, timeout=60) -> Dict[str, Any]:
    r = requests.post(BASE, json={"intent": intent, "args": args or {}}, timeout=timeout)
    try:
        return r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON from agent (status {r.status_code}): {r.text[:500]}")

def call_ok(intent: str, args: Dict[str, Any] | None = None, ctx: str = "", max_retries: int = 5, base_sleep: float = 0.6):
    """Call the agent with exponential backoff if we see rate-limit-like failures."""
    attempt = 0
    while True:
        resp = agent_call(intent, args)
        if resp.get("ok", False):
            return resp
        # Retry on likely rate-limit or transient network issues
        msg = json.dumps(resp.get("error") or {}, ensure_ascii=False)
        if "429" in msg or "HTTP failed" in msg or "INTERNAL" in msg:
            if attempt >= max_retries:
                raise RuntimeError(f"Agent error (exhausted retries) {ctx} -> {msg}")
            sleep_s = base_sleep * (2 ** attempt)
            # jitter
            sleep_s += 0.1 * attempt
            time.sleep(sleep_s)
            attempt += 1
            continue
        # Non-retriable agent error
        raise RuntimeError(f"Agent error {ctx} -> {msg}")

print("Agent:", BASE)

In [ ]:
# Soft limits to avoid 429s
MAX_LEAGUES = 12          # scan first N leagues
SLEEP_BETWEEN = 0.2       # gentle throttle between requests

leagues_resp = call_ok("leagues.list", {}, "leagues.list")
LEAGUES = leagues_resp["data"].get("leagues") or []
print("Total soccer leagues available:", len(LEAGUES))
LEAGUES = LEAGUES[:MAX_LEAGUES]
print("Scanning leagues:", len(LEAGUES))

for idx, L in enumerate(LEAGUES, 1):
    lid = L.get("idLeague"); lname = L.get("strLeague")
    if not lid:
        continue

    # Try detail, but it's optional (we print base links even if detail fails retries)
    detail = None
    try:
        detail = call_ok("league.get", {"leagueId": lid}, f"league.get {lid}")
        league_obj = detail["data"].get("league") or {}
    except Exception as e:
        league_obj = {}
        print(f"[league.get WARN] {lid} {lname}: {e}")

    vids = pick_video_fields({**L, **league_obj})
    if vids:
        print(f"\n[LEAGUE] {lname} ({lid}) — {len(vids)} link(s)")
        for v in vids:
            print("  •", v)

    if idx % 4 == 0:
        print(f"[leagues] processed {idx}/{len(LEAGUES)}")
    time.sleep(SLEEP_BETWEEN)

In [ ]:
MAX_TEAMS_PER_LEAGUE = 8   # limit teams per league
SLEEP_BETWEEN = 0.2

for li, L in enumerate(LEAGUES, 1):
    lid = L.get("idLeague"); lname = L.get("strLeague")
    if not lid:
        continue

    teams_resp = call_ok("teams.list", {"leagueId": lid}, f"teams.list {lid}")
    teams = teams_resp["data"].get("teams") or []
    teams = teams[:MAX_TEAMS_PER_LEAGUE]
    print(f"\n[LEAGUE {lname}] teams to scan: {len(teams)}")

    for T in teams:
        tid = T.get("idTeam"); tname = T.get("strTeam")
        if not tid:
            continue

        # Try detail, but fallback to list object if rate-limited
        team_obj = {}
        try:
            tdetail = call_ok("team.get", {"teamId": tid}, f"team.get {tid}")
            team_obj = tdetail["data"].get("team") or {}
        except Exception as e:
            print(f"[team.get WARN] {tname} ({tid}): {e}")

        vids = pick_video_fields({**T, **team_obj})
        if vids:
            print(f"  [TEAM] {tname} ({tid}) — {len(vids)} link(s)")
            for v in vids:
                print("    •", v)

        time.sleep(SLEEP_BETWEEN)

    if li % 3 == 0:
        print(f"[teams] leagues processed {li}/{len(LEAGUES)}")
    time.sleep(SLEEP_BETWEEN)

In [ ]:
# --- Self-contained EVENTS scan via your agent ---

import requests, time, re, json
from typing import Dict, Any, List

BASE = "http://127.0.0.1:8000/collect"  # your FastAPI agent

VIDEO_PAT = re.compile(r"(youtu\.be|youtube\.com|vimeo\.com|dai\.ly|dailymotion\.com|\.m3u8(\?|$)|\.mp4(\?|$)|\.mov(\?|$)|\.webm(\?|$))", re.I)

def pick_video_fields(obj: dict) -> List[str]:
    out = []
    for k, v in (obj or {}).items():
        if isinstance(v, str) and "http" in v and VIDEO_PAT.search(v):
            out.append(v.strip())
    # de-dup preserving order
    seen, uniq = set(), []
    for u in out:
        if u not in seen:
            uniq.append(u); seen.add(u)
    return uniq

def agent_call(intent: str, args: Dict[str, Any] | None = None, timeout=60) -> Dict[str, Any]:
    r = requests.post(BASE, json={"intent": intent, "args": args or {}}, timeout=timeout)
    try:
        return r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON from agent (status {r.status_code}): {r.text[:500]}")

def call_ok(intent: str, args: Dict[str, Any] | None = None, ctx: str = "", max_retries: int = 5, base_sleep: float = 0.6):
    """Call agent with exponential backoff for transient/429 errors."""
    attempt = 0
    while True:
        resp = agent_call(intent, args)
        if resp.get("ok", False):
            return resp
        msg = json.dumps(resp.get("error") or {}, ensure_ascii=False)
        # Retry on likely transient issues
        if "429" in msg or "HTTP failed" in msg or "INTERNAL" in msg:
            if attempt >= max_retries:
                raise RuntimeError(f"Agent error (exhausted retries) {ctx} -> {msg}")
            sleep_s = base_sleep * (2 ** attempt) + 0.1 * attempt
            time.sleep(sleep_s)
            attempt += 1
            continue
        # Non-retriable
        raise RuntimeError(f"Agent error {ctx} -> {msg}")

# Fetch leagues first (so LEAGUES is defined in this cell)
leagues_resp = call_ok("leagues.list", {}, "leagues.list")
LEAGUES: List[dict] = leagues_resp["data"].get("leagues") or []
print("Total soccer leagues available:", len(LEAGUES))

# --- CONFIG: tweak limits to avoid rate-limit ---
SEASONS_LIMIT = 1            # scan last N seasons per league
MAX_LEAGUES = 8              # scan only the first N leagues
MAX_EVENTS_PER_SEASON = 15   # cap events per season
SLEEP_BETWEEN = 0.25         # throttle between calls

LEAGUES = LEAGUES[:MAX_LEAGUES]
print("Scanning leagues:", len(LEAGUES))

def seasons_for_league(league_id: str) -> List[str]:
    sresp = call_ok("seasons.list", {"leagueId": league_id}, f"seasons.list {league_id}")
    seasons = sresp["data"].get("seasons") or []
    return [s.get("strSeason") for s in seasons if s.get("strSeason")]

for li, L in enumerate(LEAGUES, 1):
    lid = L.get("idLeague"); lname = L.get("strLeague")
    if not lid:
        continue

    all_seasons = seasons_for_league(lid)
    if not all_seasons:
        continue
    chosen = all_seasons[-SEASONS_LIMIT:] if len(all_seasons) >= SEASONS_LIMIT else all_seasons

    for season in chosen:
        events_resp = call_ok("events.list", {"leagueId": lid, "season": season},
                              f"events.list {lid} {season}")
        events = events_resp["data"].get("events") or []
        events = events[:MAX_EVENTS_PER_SEASON]

        print(f"\n[LEAGUE {lname}] season {season} — scanning {len(events)} event(s)")
        for E in events:
            eid = E.get("idEvent")
            if not eid:
                continue

            # scan base row first
            base_urls = pick_video_fields(E)
            urls = set(base_urls)

            # pull details only if base had no links (to save calls)
            if not urls:
                try:
                    edetail = call_ok("event.get", {"eventId": eid, "expand": ["timeline","stats","lineup"]},
                                      f"event.get {eid}")
                    event_obj = edetail["data"].get("event") or {}
                    timeline = edetail["data"].get("timeline") or []
                    stats = edetail["data"].get("stats") or []
                    lineup = edetail["data"].get("lineup") or []

                    for u in pick_video_fields(event_obj):
                        urls.add(u)
                    for row in timeline:
                        for u in pick_video_fields(row):
                            urls.add(u)
                    for row in stats:
                        for u in pick_video_fields(row):
                            urls.add(u)
                    for row in lineup:
                        for u in pick_video_fields(row):
                            urls.add(u)
                except Exception as e:
                    print(f"[event.get WARN] {eid}: {e}")

            if urls:
                label = E.get("strEvent") or eid
                print(f"  [EVENT] {label} ({eid}) — {len(urls)} link(s)")
                for v in sorted(urls):
                    print("    •", v)

            time.sleep(SLEEP_BETWEEN)

    if li % 2 == 0:
        print(f"[events] leagues processed {li}/{len(LEAGUES)}")
    time.sleep(SLEEP_BETWEEN)

In [ ]:
# Cell 1 — helpers (run this first)
import time, requests, json
from typing import Dict, Any, List

BASE = "https://www.thesportsdb.com/api/v1/json/3"

def GET(path: str, params: Dict[str, Any]) -> Dict[str, Any]:
    """Minimal, robust GET with cache-buster and useful debug prints."""
    p = {**params, "_ts": f"{time.time():.6f}"}  # cache‑buster
    url = f"{BASE}/{path}"
    r = requests.get(url, params=p, timeout=20)
    ctype = r.headers.get("content-type", "")
    print(f"GET {url} params={params} -> {r.status_code} {ctype}")
    r.raise_for_status()
    # Guard: sometimes the service returns HTML (e.g., splash/rate-limit)
    if "json" not in ctype:
        preview = (r.text or "")[:120].replace("\n", " ")
        raise RuntimeError(f"Non-JSON body (status {r.status_code}, ctype {ctype}) preview='{preview}'")
    return r.json()

def print_kv(obj: Dict[str, Any], keys: List[str]):
    for k in keys:
        if k in obj and obj[k] not in (None, "", []):
            print(f"{k:>14}: {obj[k]}")

In [ ]:
# Cell 2 — TEAM sanity checks
from typing import Dict, Any
import pandas as pd

def team_by_name(team_name: str) -> Dict[str, Any]:
    t = (GET("searchteams.php", {"t": team_name}).get("teams") or [None])[0] or {}
    if not t:
        raise ValueError(f"No team found for name '{team_name}'")
    tid = t.get("idTeam")
    # Cross-check lookup by ID too
    t2 = (GET("lookupteam.php", {"id": tid}).get("teams") or [None])[0] or {}
    print(f"\n=== {team_name} → id={tid}")
    keep = ["idTeam","strTeam","strLeague","idLeague","strCountry","intFormedYear","strStadium","strWebsite","strYoutube"]
    print_kv(t2 or t, keep)
    return t2 or t

def teams_by_league_name(league_name: str):
    # Prefer name-based lookup; try with sport filter then fallback
    try:
        data = GET("search_all_teams.php", {"l": league_name, "s": "Soccer"})
    except Exception as e:
        print("[WARN] name+sport failed, retrying without sport:", e)
        data = GET("search_all_teams.php", {"l": league_name})
    rows = data.get("teams") or []
    print(f"\n=== Teams in '{league_name}' — {len(rows)} rows")
    # Print a compact list like your UI
    for r in rows[:30]:
        print("-", r.get("strTeam"), "| id", r.get("idTeam"))
    return rows

# EXAMPLE — change to whatever you’re testing:
_ = teams_by_league_name("English Premier League")
_ = teams_by_league_name("German Bundesliga")
_ = team_by_name("Arsenal")

In [1]:
import time, requests, pprint, pandas as pd

BASE = "https://www.thesportsdb.com/api/v1/json/3"

def GET(path, params=None):
    p = dict(params or {})
    p["_ts"] = f"{time.time():.6f}"  # cache-buster
    r = requests.get(f"{BASE}/{path}", params=p, timeout=20)
    r.raise_for_status()
    return r.json()

def head_rows(items, n=5, cols=None):
    if not items:
        print("(empty)")
        return
    df = pd.DataFrame(items)
    if cols:
        df = df[ [c for c in cols if c in df.columns] ]
    display(df.head(n))

In [17]:
import requests

AGENT_URL = "http://127.0.0.1:8000/collect"

# Build request to our agent
payload = {
    "intent": "countries.list",
    "args": {}
}

resp = requests.post(AGENT_URL, json=payload, timeout=20)
data = resp.json()

print("ok:", data.get("ok"))
countries = data.get("data", {}).get("countries") or []
print("Total countries:", len(countries))

# Show first 10 entries raw
for c in countries[:10]:
    print(c)

ok: True
Total countries: 256
{'name_en': 'Andorra', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Andorra.png'}
{'name_en': 'United Arab Emirates', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/United-Arab-Emirates.png'}
{'name_en': 'Afghanistan', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Afghanistan.png'}
{'name_en': 'Antigua and Barbuda', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Antigua-and-Barbuda.png'}
{'name_en': 'Anguilla', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Anguilla.png'}
{'name_en': 'Albania', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Albania.png'}
{'name_en': 'Armenia', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Armenia.png'}
{'name_en': 'Angola', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Angola.png'}
{'name_en': 'Antarctica', 'flag_url_32':

In [29]:
# Cell 1 — minimal helper to call the agent
import requests, json

AGENT_URL = "http://127.0.0.1:8000/collect"  # change if different

def call_agent(intent, args=None):
    payload = {"intent": intent, "args": args or {}}
    r = requests.post(AGENT_URL, json=payload, timeout=30)
    r.raise_for_status()
    resp = r.json()
    # quick guard so failures are obvious
    if not resp.get("ok", False):
        raise RuntimeError(json.dumps(resp, indent=2))
    return resp

In [30]:
# Cell 2 — league.table test (name → id resolve happens inside the agent)
LEAGUE_NAME = "English Premier League"   # change
SEASON      = "2014-2015"                # change to any valid season for the league

resp = call_agent("league.table", {"leagueName": LEAGUE_NAME, "season": SEASON})
table = resp["data"].get("table") or []
print(f"Table rows for {LEAGUE_NAME} {SEASON}: {len(table)}")

# show first few rows raw
for row in table[:5]:
    print({k: row.get(k) for k in list(row.keys())[:10]})  # first ~10 fields

Table rows for English Premier League 2014-2015: 20
{'idStanding': '244577', 'intRank': '1', 'idTeam': '133610', 'strTeam': 'Chelsea', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/yvwvtu1448813215.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015', 'strForm': 'WLDWW', 'strDescription': 'UEFA Champions League'}
{'idStanding': '244578', 'intRank': '2', 'idTeam': '133613', 'strTeam': 'Manchester City', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/vwpvry1467462651.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015', 'strForm': 'WWWWW', 'strDescription': 'UEFA Champions League'}
{'idStanding': '244579', 'intRank': '3', 'idTeam': '133604', 'strTeam': 'Arsenal', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/uyhbfe1612467038.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015', 'strForm': 'WDDLW', 'strDescriptio

In [32]:
# Cell 2 — test team.equipment (by name, and by id)
TEAM_NAME = "Arsenal"     # change to any club
TEAM_ID   = None          # optionally set an id like "133604"

# --- by team name ---
resp_name = call_agent("team.equipment", {"teamName": TEAM_NAME})
equip_name = resp_name["data"].get("equipment") or []
print(f"[by name] ok={resp_name['ok']} | rows={len(equip_name)} | args_resolved={resp_name.get('args_resolved')}")
if equip_name:
    first = equip_name[0]
    print(" sample keys:", list(first.keys())[:12])
    print(" sample row :", {k: first.get(k) for k in list(first.keys())[:8]})

# --- by team id (if provided or from previous call) ---
if TEAM_ID is None and resp_name.get("args_resolved", {}).get("teamId"):
    TEAM_ID = resp_name["args_resolved"]["teamId"]

if TEAM_ID:
    resp_id = call_agent("team.equipment", {"teamId": str(TEAM_ID)})
    equip_id = resp_id["data"].get("equipment") or []
    print(f"\n[by id] ok={resp_id['ok']} | rows={len(equip_id)} | args_resolved={resp_id.get('args_resolved')}")
    if equip_id:
        first = equip_id[0]
        print(" sample keys:", list(first.keys())[:12])
        print(" sample row :", {k: first.get(k) for k in list(first.keys())[:8]})
else:
    print("\n[by id] skipped (no TEAM_ID available)")

[by name] ok=True | rows=142 | args_resolved={'teamId': '133604', 'teamName': 'Arsenal'}
 sample keys: ['idEquipment', 'idTeam', 'date', 'strSeason', 'strEquipment', 'strType', 'strUsername']
 sample row : {'idEquipment': '592', 'idTeam': '133604', 'date': '2019-08-15 11:41:11', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/2019-133604-Jersey.png', 'strType': '1st', 'strUsername': 'Anonymous'}

[by id] ok=True | rows=142 | args_resolved={'teamId': '133604'}
 sample keys: ['idEquipment', 'idTeam', 'date', 'strSeason', 'strEquipment', 'strType', 'strUsername']
 sample row : {'idEquipment': '592', 'idTeam': '133604', 'date': '2019-08-15 11:41:11', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/2019-133604-Jersey.png', 'strType': '1st', 'strUsername': 'Anonymous'}


In [6]:
PLAYER_NAME = "Lionel Messi"  # change as needed

def resolve_player_id(player_name: str) -> str:
    players = GET("searchplayers.php", {"p": player_name}).get("player") or []
    if not players:
        raise ValueError(f"No player found: {player_name}")
    name_l = player_name.strip().lower()
    exact = [p for p in players if (p.get("strPlayer") or "").strip().lower()==name_l]
    pick = exact[0] if exact else players[0]
    return str(pick["idPlayer"])

PID = resolve_player_id(PLAYER_NAME)

honours   = GET("lookuphonours.php",      {"id": PID}).get("honours") or []
former    = GET("lookupformerteams.php",  {"id": PID}).get("formerteams") or []
milestone = GET("lookupmilestones.php",   {"id": PID}).get("milestones") or []
contracts = GET("lookupcontracts.php",    {"id": PID}).get("contracts") or []
pres      = GET("playerresults.php",      {"id": PID}).get("results") or []

print(f"[{PLAYER_NAME}] id={PID}")
print("Honours  :", len(honours));   head_rows(honours, n=6)
print("Former   :", len(former));    head_rows(former, n=6)
print("Milestone:", len(milestone)); head_rows(milestone, n=6)
print("Contracts:", len(contracts)); head_rows(contracts, n=6)
print("Results  :", len(pres));      head_rows(pres, n=6)

[Lionel Messi] id=34146370
Honours  : 129


,id,idPlayer,idTeam,idLeague,idHonour,strSport,strPlayer,strTeam,strTeamBadge,strHonour,strHonourLogo,strHonourTrophy,strSeason
0,611,34146370,133739,4335,566,Soccer,Lionel Messi,Barcelona,https://r2.thesportsdb.com/images/media/team/b...,Spanish La Liga,https://www.thesportsdb.com/images/media/honou...,https://www.thesportsdb.com/images/media/honou...,2004-2005
1,612,34146370,133739,4335,566,Soccer,Lionel Messi,Barcelona,https://r2.thesportsdb.com/images/media/team/b...,Spanish La Liga,https://www.thesportsdb.com/images/media/honou...,https://www.thesportsdb.com/images/media/honou...,2005-2006
2,613,34146370,133739,4335,566,Soccer,Lionel Messi,Barcelona,https://r2.thesportsdb.com/images/media/team/b...,Spanish La Liga,https://www.thesportsdb.com/images/media/honou...,https://www.thesportsdb.com/images/media/honou...,2008-2009
3,614,34146370,133739,4335,566,Soccer,Lionel Messi,Barcelona,https://r2.thesportsdb.com/images/media/team/b...,Spanish La Liga,https://www.thesportsdb.com/images/media/honou...,https://www.thesportsdb.com/images/media/honou...,2009-2010
4,615,34146370,133739,4335,566,Soccer,Lionel Messi,Barcelona,https://r2.thesportsdb.com/images/media/team/b...,Spanish La Liga,https://www.thesportsdb.com/images/media/honou...,https://www.thesportsdb.com/images/media/honou...,2010-2011
5,616,34146370,133739,4335,566,Soccer,Lionel Messi,Barcelona,https://r2.thesportsdb.com/images/media/team/b...,Spanish La Liga,https://www.thesportsdb.com/images/media/honou...,https://www.thesportsdb.com/images/media/honou...,2012-2013


Former   : 8


,id,idPlayer,idFormerTeam,strSport,strPlayer,strFormerTeam,strMoveType,strBadge,strJoined,strDeparted
0,1142,34146370,134487,Soccer,Lionel Messi,Barcelona B,Permanent,https://r2.thesportsdb.com/images/media/team/b...,2004,2005
1,21462,34146370,133739,Soccer,Lionel Messi,Barcelona,Youth,https://r2.thesportsdb.com/images/media/team/b...,2001,2004
2,26234,34146370,135166,Soccer,Lionel Messi,Newells Old Boys,Youth,https://r2.thesportsdb.com/images/media/team/b...,1994,2000
3,32956,34146370,134509,Soccer,Lionel Messi,Argentina,Permanent,https://r2.thesportsdb.com/images/media/team/b...,2005,
4,44898,34146370,133739,Soccer,Lionel Messi,Barcelona,Permanent,https://r2.thesportsdb.com/images/media/team/b...,2004,2021
5,51256,34146370,143164,Soccer,Lionel Messi,Argentina U23,Permanent,https://r2.thesportsdb.com/images/media/team/b...,2008,2008


Milestone: 15


,id,idPlayer,strPlayer,idTeam,idMilestone,strTeam,strSport,strMilestone,strMilestoneLogo,dateMilestone
0,92,34146370,Lionel Messi,134509,1676,Argentina,Soccer,FIFA World Cup Dream Team,https://www.thesportsdb.com/images/media/miles...,2014-07-15
1,1347,34146370,Lionel Messi,134509,1843,Argentina,Soccer,FIFA World Cup Player Of The Match,https://www.thesportsdb.com/images/media/miles...,2018-06-26
2,1459,34146370,Lionel Messi,134509,1843,Argentina,Soccer,FIFA World Cup Player Of The Match,https://www.thesportsdb.com/images/media/miles...,2022-11-26
3,1522,34146370,Lionel Messi,134509,1843,Argentina,Soccer,FIFA World Cup Player Of The Match,https://www.thesportsdb.com/images/media/miles...,2022-12-03
4,1579,34146370,Lionel Messi,134509,1843,Argentina,Soccer,FIFA World Cup Player Of The Match,https://www.thesportsdb.com/images/media/miles...,2022-12-09
5,1593,34146370,Lionel Messi,134509,1843,Argentina,Soccer,FIFA World Cup Player Of The Match,https://www.thesportsdb.com/images/media/miles...,2022-12-13


Contracts: 3


,id,idPlayer,idTeam,strSport,strPlayer,strTeam,strBadge,strYearStart,strYearEnd,strWage
0,238,34146370,133739,Soccer,Lionel Messi,Barcelona,https://r2.thesportsdb.com/images/media/team/b...,2005,2021,
1,2970,34146370,133714,Soccer,Lionel Messi,Paris SG,https://r2.thesportsdb.com/images/media/team/b...,2021,2023,
2,5035,34146370,137699,Soccer,Lionel Messi,Inter Miami,https://r2.thesportsdb.com/images/media/team/b...,2023,2025,


Results  : 0
(empty)


In [7]:
EVENT_NAME = "Standard vs Charleroi"   # change to any match-up string you know

# 1) Find event candidates by name
cands = GET("searchevents.php", {"e": EVENT_NAME}).get("event") or []
print(f"Candidates: {len(cands)}")
head_rows(cands, n=6, cols=["idEvent","dateEvent","strSeason","strEvent","strLeague"])

# 2) Pick one (by index or filter by season/date)
ev = cands[0] if cands else {}
EID = ev.get("idEvent")
print("Chosen event id:", EID, "|", ev.get("strEvent"), ev.get("dateEvent"), ev.get("strSeason"))

# 3) Event results (per-event stats/notes history list, if present)
eres = GET("eventresults.php", {"id": EID}).get("results") or []
print("eventresults rows:", len(eres))
head_rows(eres, n=10)

# 4) Event TV (broadcasters) – direct by id
etv = GET("lookuptv.php", {"id": EID}).get("tvevent") or []
print("lookuptv rows:", len(etv))
head_rows(etv, n=10)

# (Optional) TV schedule by day/league/sport:
# tv_day = GET("eventstv.php", {"d": "2014-07-25", "s": "Soccer", "l": "Belgian Pro League"}).get("tvevents") or []
# print("eventstv rows:", len(tv_day)); head_rows(tv_day, n=10)

Candidates: 14


,idEvent,dateEvent,strSeason,strEvent,strLeague
0,2239080,2025-05-04,2024-2025,Standard vs Charleroi,Belgian Pro League
1,2076917,2024-10-20,2024-2025,Standard vs Charleroi,Belgian Pro League
2,1823946,2023-12-16,2023-2024,Standard vs Charleroi,Belgian First Division A
3,1578244,2023-04-14,2022-2023,Standard vs Charleroi,Belgian First Division A
4,1245541,2021-12-05,2021-2022,Standard vs Charleroi,Belgian First Division A
5,1015182,2021-01-24,2020-2021,Standard vs Charleroi,Belgian First Division A


Chosen event id: 2239080 | Standard vs Charleroi 2025-05-04 2024-2025
eventresults rows: 0
(empty)
lookuptv rows: 2


,id,idEvent,intDivision,strSport,strEvent,strEventThumb,strEventPoster,strEventBanner,strEventSquare,idChannel,strCountry,strEventCountry,strLogo,strChannel,strSeason,strTime,dateEvent,strTimeStamp
0,4867,584911,None,Motorsport,Marrakesh E-Prix,None,None,None,None,8165,Ireland,None,https://r2.thesportsdb.com/images/media/channe...,BT Sport ESPN HD,1819,16:00:00,2019-01-12,None
1,6021,584911,None,Motorsport,Marrakesh E-Prix,None,None,None,None,8163,Ireland,None,https://r2.thesportsdb.com/images/media/channe...,BT Sport ESPN,1819,16:00:00,2019-01-12,None


In [10]:
import time, requests, pandas as pd

BASE = "https://www.thesportsdb.com/api/v1/json/3"

def get_json_safe(path, params=None, show_head=True):
    p = dict(params or {}); p["_ts"] = f"{time.time():.6f}"
    r = requests.get(f"{BASE}/{path}", params=p, timeout=20)
    ct = r.headers.get("content-type","").lower()
    try:
        j = r.json()
        return j, {"ok": True, "status": r.status_code, "ct": ct, "url": r.url}
    except Exception as e:
        head = r.text[:240].replace("\n"," ") if show_head else ""
        return {}, {"ok": False, "status": r.status_code, "ct": ct, "url": r.url, "err": str(e), "head": head}

# --- Parameters you can tweak ---
HL_DATE   = "2014-07-25"            # YYYY-MM-DD
HL_SPORT  = "Soccer"                # optional
HL_LEAGUE = "Belgian Pro League"    # optional (try removing if empty)

# 1) Try eventshighlights.php (primary)
j, meta = get_json_safe("eventshighlights.php", {"d": HL_DATE, "s": HL_SPORT, "l": HL_LEAGUE})
highlights = j.get("highlights") or []

print("[eventshighlights] ok:", meta["ok"], "| status:", meta["status"])
print("URL:", meta["url"])
if not meta["ok"]:
    print("Content-Type:", meta["ct"])
    print("Body head:", meta.get("head",""))

if highlights:
    print(f"\nHighlights via eventshighlights.php: {len(highlights)}")
    df = pd.DataFrame(highlights)
    cols = [c for c in ["strEvent","strLeague","dateEvent","strVideo","strThumb"] if c in df.columns]
    display(df[cols].head(15))
else:
    # 2) Fallback: scan the day's events for embedded strVideo links
    print("\nNo JSON highlights. Falling back to eventsday.php and collecting strVideo links…")
    j2, meta2 = get_json_safe("eventsday.php", {"d": HL_DATE, "s": HL_SPORT})
    evs = j2.get("events") or []
    # If you want to constrain to a specific league name, filter here:
    if HL_LEAGUE:
        evs = [e for e in evs if (e.get("strLeague") or "").strip().lower() == HL_LEAGUE.strip().lower()]
    vids = [e for e in evs if (e.get("strVideo") or "").strip()]
    print(f"[eventsday] events={len(evs)} | with videos={len(vids)} | URL: {meta2['url']}")
    if vids:
        dfv = pd.DataFrame(vids)
        cols = [c for c in ["idEvent","strEvent","strLeague","dateEvent","strVideo"] if c in dfv.columns]
        display(dfv[cols].head(25))
    else:
        print("No embedded video links found for that date/filters.")

[eventshighlights] ok: False | status: 200
URL: https://www.thesportsdb.com/api/v1/json/3/eventshighlights.php?d=2014-07-25&s=Soccer&l=Belgian+Pro+League&_ts=1756214336.576732
Content-Type: text/html; charset=utf-8
Body head: 

No JSON highlights. Falling back to eventsday.php and collecting strVideo links…
[eventsday] events=0 | with videos=0 | URL: https://www.thesportsdb.com/api/v1/json/3/eventsday.php?d=2014-07-25&s=Soccer&_ts=1756214337.011039
No embedded video links found for that date/filters.


In [9]:
# Option A: from a team
TEAM_NAME = "Arsenal"
tid = (GET("searchteams.php", {"t": TEAM_NAME}).get("teams") or [])[0]["idTeam"]
team_full = (GET("lookupteam.php", {"id": tid}).get("teams") or [None])[0] or {}
VENUE_ID = team_full.get("idVenue")

print("Team venue id:", VENUE_ID)
venue = (GET("lookupvenue.php", {"id": VENUE_ID}).get("venues") or [None])[0] or {}
pprint.pprint({k: venue.get(k) for k in ["idVenue","strVenue","strCountry","intCapacity","strLocation","strWebsite"]})

# Option B: from an event
# ev = (GET("searchevents.php", {"e": "Liverpool vs Swansea"}).get("event") or [None])[0] or {}
# VID = ev.get("idVenue")
# print("Event venue id:", VID)
# v2 = (GET("lookupvenue.php", {"id": VID}).get("venues") or [None])[0] or {}
# pprint.pprint({k: v2.get(k) for k in ["idVenue","strVenue","strCountry","intCapacity","strLocation","strWebsite"]})

Team venue id: 15528
{'idVenue': '15528',
 'intCapacity': '60338',
 'strCountry': 'England',
 'strLocation': 'Holloway, London, England',
 'strVenue': 'Emirates Stadium',
 'strWebsite': 'www.arsenal.com/the-club/emirates-stadium'}


In [33]:
resp = call_agent("sports.list", {})
sports = resp["data"].get("sports") or []
print("sports:", len(sports))
print(sports[:3])  # raw peek

sports: 1
[{'idSport': '102', 'strSport': 'Soccer', 'strFormat': 'TeamvsTeam', 'strSportThumb': 'https://www.thesportsdb.com/images/sports/soccer.jpg', 'strSportThumbBW': 'https://www.thesportsdb.com/images/sports/bw/soccer.jpg', 'strSportIconGreen': 'https://www.thesportsdb.com/images/icons/sports/soccer.png', 'strSportDescription': "Association football, more commonly known as football or soccer, is a team sport played between two teams of eleven players with a spherical ball. It is played by 250 million players in over 200 countries and dependencies, making it the world's most popular sport. The game is played on a rectangular field with a goal at each end. The object of the game is to score by getting the ball into the opposing goal.\r\n\r\nPlayers are not allowed to touch the ball with their hands or arms while it is in play, unless they are goalkeepers (and then only when within their penalty area). Other players mainly use their feet to strike or pass the ball, but may also use 

In [34]:
resp = call_agent("countries.list", {})
# agent returns {"countries": [...]} or {"raw": <upstream>}, depending on upstream shape
payload = resp["data"]
if "countries" in payload:
    countries = payload["countries"] or []
    print("countries:", len(countries))
    print(countries[:5])
else:
    print("raw top-level keys:", list(payload.get("raw", {}).keys()))

countries: 256
[{'name_en': 'Andorra', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Andorra.png'}, {'name_en': 'United Arab Emirates', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/United-Arab-Emirates.png'}, {'name_en': 'Afghanistan', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Afghanistan.png'}, {'name_en': 'Antigua and Barbuda', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Antigua-and-Barbuda.png'}, {'name_en': 'Anguilla', 'flag_url_32': 'https://www.thesportsdb.com/images/icons/flags/shiny/32/Anguilla.png'}]


In [35]:
LEAGUE_NAME = "English Premier League"   # change as needed
SEASON      = "2014-2015"                # must exist for league

resp = call_agent("league.table", {"leagueName": LEAGUE_NAME, "season": SEASON})
table = resp["data"].get("table") or []
print(f"rows: {len(table)} | sample:")
for row in table[:3]:
    # show the first ~10 keys to see shape (position, name, P/W/D/L/GD/pts, etc.)
    keys = list(row.keys())[:10]
    print({k: row.get(k) for k in keys})

rows: 20 | sample:
{'idStanding': '244577', 'intRank': '1', 'idTeam': '133610', 'strTeam': 'Chelsea', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/yvwvtu1448813215.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015', 'strForm': 'WLDWW', 'strDescription': 'UEFA Champions League'}
{'idStanding': '244578', 'intRank': '2', 'idTeam': '133613', 'strTeam': 'Manchester City', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/vwpvry1467462651.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015', 'strForm': 'WWWWW', 'strDescription': 'UEFA Champions League'}
{'idStanding': '244579', 'intRank': '3', 'idTeam': '133604', 'strTeam': 'Arsenal', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/uyhbfe1612467038.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015', 'strForm': 'WDDLW', 'strDescription': 'UEFA Champions League'}


In [36]:
# Use either teamName or teamId
TEAM_NAME = "Arsenal"

resp = call_agent("team.equipment", {"teamName": TEAM_NAME})
equip = resp["data"].get("equipment") or []
print(f"equipment rows for {TEAM_NAME}:", len(equip))
print(equip[:2])  # raw peek

equipment rows for Arsenal: 142
[{'idEquipment': '592', 'idTeam': '133604', 'date': '2019-08-15 11:41:11', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/2019-133604-Jersey.png', 'strType': '1st', 'strUsername': 'Anonymous'}, {'idEquipment': '1130', 'idTeam': '133604', 'date': '2020-04-14 15:27:09', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/svssgp1586874381.png', 'strType': '2nd', 'strUsername': 'cydalby'}]


In [37]:
PLAYER_NAME = "Lionel Messi"

resp = call_agent("player.honours", {"playerName": PLAYER_NAME})
honours = resp["data"].get("honours") or []
print(f"honours for {PLAYER_NAME}:", len(honours))
print(honours[:3])

honours for Lionel Messi: 129
[{'id': '611', 'idPlayer': '34146370', 'idTeam': '133739', 'idLeague': '4335', 'idHonour': '566', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strTeam': 'Barcelona', 'strTeamBadge': 'https://r2.thesportsdb.com/images/media/team/badge/wq9sir1639406443.png', 'strHonour': 'Spanish La Liga', 'strHonourLogo': 'https://www.thesportsdb.com/images/media/honour/logo/f83x5q1725095441.png', 'strHonourTrophy': 'https://www.thesportsdb.com/images/media/honour/trophy/nco20w1650642293.png', 'strSeason': '2004-2005'}, {'id': '612', 'idPlayer': '34146370', 'idTeam': '133739', 'idLeague': '4335', 'idHonour': '566', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strTeam': 'Barcelona', 'strTeamBadge': 'https://r2.thesportsdb.com/images/media/team/badge/wq9sir1639406443.png', 'strHonour': 'Spanish La Liga', 'strHonourLogo': 'https://www.thesportsdb.com/images/media/honour/logo/f83x5q1725095441.png', 'strHonourTrophy': 'https://www.thesportsdb.com/images/media/honou

In [38]:
PLAYER_NAME = "Lionel Messi"

resp = call_agent("player.former_teams", {"playerName": PLAYER_NAME})
former = resp["data"].get("formerteams") or []
print(f"former teams for {PLAYER_NAME}:", len(former))
print(former[:3])

former teams for Lionel Messi: 8
[{'id': '1142', 'idPlayer': '34146370', 'idFormerTeam': '134487', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strFormerTeam': 'Barcelona B', 'strMoveType': 'Permanent', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/rwxxuw1420415212.png', 'strJoined': '2004', 'strDeparted': '2005'}, {'id': '21462', 'idPlayer': '34146370', 'idFormerTeam': '133739', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strFormerTeam': 'Barcelona', 'strMoveType': 'Youth', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/xqwpup1473502878.png', 'strJoined': '2001', 'strDeparted': '2004'}, {'id': '26234', 'idPlayer': '34146370', 'idFormerTeam': '135166', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strFormerTeam': 'Newells Old Boys', 'strMoveType': 'Youth', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/23aftf1580842633.png', 'strJoined': '1994', 'strDeparted': '2000'}]


In [39]:
PLAYER_NAME = "Lionel Messi"

resp = call_agent("player.milestones", {"playerName": PLAYER_NAME})
milestones = resp["data"].get("milestones") or []
print(f"milestones for {PLAYER_NAME}:", len(milestones))
print(milestones[:3])

milestones for Lionel Messi: 15
[{'id': '92', 'idPlayer': '34146370', 'strPlayer': 'Lionel Messi', 'idTeam': '134509', 'idMilestone': '1676', 'strTeam': 'Argentina', 'strSport': 'Soccer', 'strMilestone': 'FIFA World Cup Dream Team', 'strMilestoneLogo': 'https://www.thesportsdb.com/images/media/milestone/logo/2zv57r1661366427.png', 'dateMilestone': '2014-07-15'}, {'id': '1347', 'idPlayer': '34146370', 'strPlayer': 'Lionel Messi', 'idTeam': '134509', 'idMilestone': '1843', 'strTeam': 'Argentina', 'strSport': 'Soccer', 'strMilestone': 'FIFA World Cup Player Of The Match', 'strMilestoneLogo': 'https://www.thesportsdb.com/images/media/milestone/logo/kzpd281669055640.png', 'dateMilestone': '2018-06-26'}, {'id': '1459', 'idPlayer': '34146370', 'strPlayer': 'Lionel Messi', 'idTeam': '134509', 'idMilestone': '1843', 'strTeam': 'Argentina', 'strSport': 'Soccer', 'strMilestone': 'FIFA World Cup Player Of The Match', 'strMilestoneLogo': 'https://www.thesportsdb.com/images/media/milestone/logo/kzpd

In [40]:
PLAYER_NAME = "Lionel Messi"

resp = call_agent("player.milestones", {"playerName": PLAYER_NAME})
milestones = resp["data"].get("milestones") or []
print(f"milestones for {PLAYER_NAME}:", len(milestones))
print(milestones[:3])

milestones for Lionel Messi: 15
[{'id': '92', 'idPlayer': '34146370', 'strPlayer': 'Lionel Messi', 'idTeam': '134509', 'idMilestone': '1676', 'strTeam': 'Argentina', 'strSport': 'Soccer', 'strMilestone': 'FIFA World Cup Dream Team', 'strMilestoneLogo': 'https://www.thesportsdb.com/images/media/milestone/logo/2zv57r1661366427.png', 'dateMilestone': '2014-07-15'}, {'id': '1347', 'idPlayer': '34146370', 'strPlayer': 'Lionel Messi', 'idTeam': '134509', 'idMilestone': '1843', 'strTeam': 'Argentina', 'strSport': 'Soccer', 'strMilestone': 'FIFA World Cup Player Of The Match', 'strMilestoneLogo': 'https://www.thesportsdb.com/images/media/milestone/logo/kzpd281669055640.png', 'dateMilestone': '2018-06-26'}, {'id': '1459', 'idPlayer': '34146370', 'strPlayer': 'Lionel Messi', 'idTeam': '134509', 'idMilestone': '1843', 'strTeam': 'Argentina', 'strSport': 'Soccer', 'strMilestone': 'FIFA World Cup Player Of The Match', 'strMilestoneLogo': 'https://www.thesportsdb.com/images/media/milestone/logo/kzpd

In [41]:
PLAYER_NAME = "Lionel Messi"

resp = call_agent("player.contracts", {"playerName": PLAYER_NAME})
contracts = resp["data"].get("contracts") or []
print(f"contracts for {PLAYER_NAME}:", len(contracts))
print(contracts[:3])

contracts for Lionel Messi: 3
[{'id': '238', 'idPlayer': '34146370', 'idTeam': '133739', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strTeam': 'Barcelona', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/xqwpup1473502878.png', 'strYearStart': '2005', 'strYearEnd': '2021', 'strWage': ''}, {'id': '2970', 'idPlayer': '34146370', 'idTeam': '133714', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strTeam': 'Paris SG', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/rwqrrq1473504808.png', 'strYearStart': '2021', 'strYearEnd': '2023', 'strWage': ''}, {'id': '5035', 'idPlayer': '34146370', 'idTeam': '137699', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strTeam': 'Inter Miami', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/m4it3e1602103647.png', 'strYearStart': '2023', 'strYearEnd': '2025', 'strWage': ''}]


In [42]:
PLAYER_NAME = "Lionel Messi"

resp = call_agent("player.results", {"playerName": PLAYER_NAME})
results = resp["data"].get("results") or []
print(f"player results for {PLAYER_NAME}:", len(results))
print(results[:3])

player results for Lionel Messi: 0
[]


In [43]:
# Option A: Past for a league+season (use events.list for season; event.results is "past league" shortcut)
LEAGUE_NAME = "Belgian Pro League"

resp = call_agent("event.results", {"leagueName": LEAGUE_NAME})
evs = resp["data"].get("events") or []
print(f"past league events ({LEAGUE_NAME}):", len(evs))
print(evs[:2])

# Option B: Last events for a team
TEAM_NAME = "Arsenal"
resp2 = call_agent("event.results", {"teamName": TEAM_NAME})
team_last = resp2["data"].get("events") or []
print(f"\nlast team events ({TEAM_NAME}):", len(team_last))
print(team_last[:2])

past league events (Belgian Pro League): 15
[{'idEvent': '2274693', 'idAPIfootball': '1387162', 'strEvent': 'Luton Town vs Cardiff City', 'strEventAlternate': 'Cardiff City @ Luton Town', 'strFilename': 'English League 1 2025-08-23 Luton Town vs Cardiff City', 'strSport': 'Soccer', 'idLeague': '4396', 'strLeague': 'English League 1', 'strLeagueBadge': 'https://r2.thesportsdb.com/images/media/league/badge/afedb31688770443.png', 'strSeason': '2025-2026', 'strDescriptionEN': '', 'strHomeTeam': 'Luton Town', 'strAwayTeam': 'Cardiff City', 'intHomeScore': '0', 'intRound': '5', 'intAwayScore': '1', 'intSpectators': None, 'strOfficial': '', 'strTimestamp': '2025-08-23T11:30:00', 'dateEvent': '2025-08-23', 'dateEventLocal': '2025-08-23', 'strTime': '11:30:00', 'strTimeLocal': '12:30:00', 'strGroup': '', 'idHomeTeam': '133888', 'strHomeTeamBadge': 'https://r2.thesportsdb.com/images/media/team/badge/v977eh1681319466.png', 'idAwayTeam': '133637', 'strAwayTeamBadge': 'https://r2.thesportsdb.com/im

In [44]:
# Name-first; optional eventId filter if you know it
EVENT_NAME = "Liverpool vs Swansea"
resp = call_agent("video.highlights", {"eventName": EVENT_NAME})
vids = resp["data"].get("videos") or []
print(f"video highlights for '{EVENT_NAME}':", len(vids))
print(vids[:3])

video highlights for 'Liverpool vs Swansea': 0
[]


In [50]:
TEAM_NAME = "Blackpool"  # change as needed

resp = call_agent("players.list", {"teamName": TEAM_NAME})
players = resp["data"].get("players") or []
print(f"Players for '{TEAM_NAME}': {len(players)}")
for p in players[:12]:  # sample
    print("-", p.get("idPlayer"), "|", p.get("strPlayer"), "|", p.get("strPosition"), "|", p.get("strTeam"))

Players for 'Blackpool': 30
- 34175641 | Albert Sambi Lokonga | Central Midfield | Arsenal
- 34163698 | Ben White | Right-Back | Arsenal
- 34169884 | Bukayo Saka | Right Winger | Arsenal
- 34164499 | Christian Nørgaard | Defensive Midfield | Arsenal
- 34194118 | Cristhian Mosquera | Centre-Back | Arsenal
- 34148681 | David Raya | Goalkeeper | Arsenal
- 34161584 | Declan Rice | Defensive Midfield | Arsenal
- 34172278 | Eberechi Eze | Attacking Midfield | Arsenal
- 34200626 | Ethan Nwaneri | Attacking Midfield | Arsenal
- 34176976 | Fábio Vieira | Attacking Midfield | Arsenal
- 34160962 | Gabriel Jesus | Centre-Forward | Arsenal
- 34172252 | Gabriel Magalhães | Centre-Back | Arsenal


In [55]:
import requests, time, random, hashlib, json

BASE = "https://www.thesportsdb.com/api/v1/json/3"

def GET(ep, params=None):
    """Raw GET with strong cache-busting and no filtering."""
    q = dict(params or {})
    # Two cache busters: _ts (float-ish) and _rnd (int)
    q["_ts"] = f"{time.time():.6f}"
    q["_rnd"] = str(random.randrange(1_000_000_000))
    r = requests.get(
        f"{BASE}/{ep}",
        params=q,
        headers={
            "Cache-Control": "no-cache",
            "Pragma": "no-cache",
            "User-Agent": "diag-notebook/players-raw"
        },
        timeout=30
    )
    # Try JSON; if it fails, show the first 160 chars of text
    try:
        j = r.json()
    except Exception:
        head = (r.text or "")[:160].replace("\n", " ")
        print(f"[{ep}] HTTP {r.status_code} non-JSON. head:", head)
        return {}
    return j or {}

def sha1_of(obj) -> str:
    s = json.dumps(obj, sort_keys=True, separators=(",",":")).encode("utf-8")
    return hashlib.sha1(s).hexdigest()

def resolve_team_id(team_name: str) -> str | None:
    j = GET("searchteams.php", {"t": team_name})
    teams = j.get("teams") or []
    if not teams:
        return None
    # prefer exact name; else first
    name_l = team_name.strip().lower()
    exact = [t for t in teams if (t.get("strTeam") or "").strip().lower()==name_l]
    pick = exact[0] if exact else teams[0]
    return str(pick.get("idTeam") or "")

def show(method_label, payload_key, raw):
    rows = raw.get(payload_key) or []
    print(f"\n— {method_label} —")
    print("rows:", len(rows))
    if rows:
        # team labels present in payload (helps detect the “Arsenal everywhere” bug)
        teams = sorted({r.get("strTeam") for r in rows})
        print("team labels:", teams[:8], f"(unique={len(teams)})")
        print("sha1:", sha1_of(rows), "len:", len(json.dumps(rows)))
        for r in rows[:8]:
            print("-", r.get("idPlayer"), "|", r.get("strPlayer"), "|", r.get("strPosition"), "|", r.get("strTeam"))
        # show the available columns
        cols = sorted({k for r in rows for k in (r.keys() if isinstance(r, dict) else [])})
        print("columns:", len(cols))
        print(cols[:20], "…")
    else:
        print("!! EMPTY (or non-JSON/Patreon)")

def diag_for_team(team_name: str):
    print("\n" + "="*80)
    print("TEAM:", team_name)
    team_id = resolve_team_id(team_name)
    print("resolved teamId:", team_id)

    # A) NAME-BASED: searchplayers.php?t=TEAM_NAME
    a = GET("searchplayers.php", {"t": team_name})
    show("A) searchplayers.php?t=<team name>", "player", a)

    # B) ID-BASED: lookup_all_players.php?id=TEAM_ID
    if team_id:
        b = GET("lookup_all_players.php", {"id": team_id})
        show("B) lookup_all_players.php?id=<team id>", "player", b)
    else:
        print("\n— B) lookup_all_players.php — skipped (no teamId)")

    # C) LINEUP-DERIVED: eventslast → pick 1st event → lookuplineup
    if team_id:
        last_ev = GET("eventslast.php", {"id": team_id})
        results = last_ev.get("results") or []
        if results:
            e = results[0].get("idEvent")
            print("last event id:", e, "|", results[0].get("strEvent"))
            lu = GET("lookuplineup.php", {"id": e})
            # lineup is a flat list with strTeam, strPlayer, strPosition, strHome/strAway, etc.
            # Keep raw but present in same “players-like” view:
            show("C) lookuplineup.php (from last event)", "lineup", lu)
        else:
            print("— C) lookuplineup — no recent events for this team")
    else:
        print("— C) lookuplineup — skipped (no teamId)")

# >>> Try a few clubs. Change or add more to stress-test.
for club in ["Arsenal", "Chelsea", "Barcelona", "Bayern Munich"]:
    diag_for_team(club)


TEAM: Arsenal
resolved teamId: 133604

— A) searchplayers.php?t=<team name> —
rows: 25
team labels: ['Arsenal'] (unique=1)
sha1: 5b62e9023ed0d787c06f7de6b01d98cc1225c0d6 len: 10818
- 34145406 | Mikel Arteta | Manager | Arsenal
- 34145922 | Raheem Sterling | Left Wing | Arsenal
- 34148681 | David Raya | Goalkeeper | Arsenal
- 34153358 | Leandro Trossard | Left Wing | Arsenal
- 34159218 | Martin Ødegaard | Attacking Midfield | Arsenal
- 34160650 | Kepa Arrizabalaga | Goalkeeper | Arsenal
- 34160933 | Oleksandr Zinchenko | Left-Back | Arsenal
- 34160962 | Gabriel Jesus | Centre-Forward | Arsenal
columns: 12
['dateBorn', 'idPlayer', 'idTeam', 'strCutout', 'strGender', 'strNationality', 'strPlayer', 'strPosition', 'strSport', 'strStatus', 'strTeam', 'strThumb'] …

— B) lookup_all_players.php?id=<team id> —
rows: 30
team labels: ['Arsenal'] (unique=1)
sha1: 54f6e9fa0426b450d96f917ecfc0a9b86ddb4854 len: 80821
- 34175641 | Albert Sambi Lokonga | Central Midfield | Arsenal
- 34163698 | Ben Whi

In [68]:
import requests, time, json

BASE = "https://www.thesportsdb.com/api/v1/json/3"

def GET(ep, params=None):
    q = dict(params or {})
    q["_ts"] = f"{time.time():.6f}"  # cache-buster
    r = requests.get(f"{BASE}/{ep}", params=q, timeout=25)
    try:
        return r.json() or {}
    except Exception:
        print(f"[{ep}] non-JSON or empty. HTTP", r.status_code, "| head:", (r.text or "")[:160].replace("\n"," "))
        return {}

def resolve_player_id(name: str) -> str | None:
    j = GET("searchplayers.php", {"p": name})
    arr = j.get("player") or []
    if not arr: return None
    name_l = name.strip().lower()
    exact = [p for p in arr if (p.get("strPlayer") or "").strip().lower()==name_l]
    pick = exact[0] if exact else arr[0]
    return str(pick.get("idPlayer") or "")

In [72]:
from pprint import pprint

def get_player_by_id(player_id: str) -> dict:
    j = GET("lookupplayer.php", {"id": player_id})
    return (j.get("players") or [None])[0] or {}

def lineup_player_names_for_team(team_name: str) -> list[str]:
    # last event → lineup → player names (works even when roster endpoints are wrong)
    tj = GET("searchteams.php", {"t": team_name})
    teams = tj.get("teams") or []
    if not teams: return []
    team_id = str(teams[0].get("idTeam"))
    last = GET("eventslast.php", {"id": team_id}).get("results") or []
    if not last: return []
    ev_id = last[0].get("idEvent")
    lu = GET("lookuplineup.php", {"id": ev_id}).get("lineup") or []
    # prefer starters; fall back to all
    starters = [r for r in lu if r.get("strSubstitute") == "No"] or lu
    names = []
    for r in starters:
        n = (r.get("strPlayer") or "").strip()
        if n and n not in names:
            names.append(n)
    return names

# # ---- Try A: direct known names -> resolve PlayerIDs -> lookup by id
# names = ["Bukayo Saka", "Declan Rice", "Gabriel Jesus"]  # change as you like
# pids = [resolve_player_id(n) for n in names]
# pids = [pid for pid in pids if pid]
# print("Resolved PlayerIDs:", pids)

# players = [get_player_by_id(pid) for pid in pids]
# for p in players:
#     print("\n—", p.get("strPlayer"), "— id:", p.get("idPlayer"))
#     for k in ["strTeam","strNationality","strPosition","dateBorn","strWage"]:
#         print(f"{k:15}", p.get(k))
#     # show a couple of IDs so you can key on them later
#     for k in ["idAPIfootball","idWikidata","idPlayer"]:
#         print(f"{k:15}", p.get(k))

# ---- Try B (optional): derive PlayerIDs from the team’s *actual* last-lineup names
team = "Arsenal"  # change team
ln_names = lineup_player_names_for_team(team)
print(f"\n[{team}] lineup-derived names ({len(ln_names)}):", ln_names[:12])

ln_pids = [resolve_player_id(n) for n in ln_names]
ln_pids = [pid for pid in ln_pids if pid]
print("lineup-derived PlayerIDs:", ln_pids[:10], "… total:", len(ln_pids))

ln_players = [get_player_by_id(pid) for pid in ln_pids[:10]]  # first 10 for brevity
for p in ln_players:
    print("\n•", p.get("strPlayer"), "| id:", p.get("idPlayer"), "| team:", p.get("strTeam"), "| pos:", p.get("strPosition"))


[Arsenal] lineup-derived names (22): ['Matty Cash', 'Tyrone Mings', 'Matt Targett', 'Douglas Luiz', 'Trézéguet', 'Ross Barkley', 'Jack Grealish', 'Ollie Watkins', 'Emiliano Martinez', 'Ezri Konsa Ngoyo', 'John McGinn', 'Adrian']
lineup-derived PlayerIDs: ['34161548', '34147598', '34145735', '34161348', '34167707', '34145837', '34145477', '34157367', '34145423', '34161727'] … total: 22
[lookupplayer.php] non-JSON or empty. HTTP 429 | head: <!DOCTYPE html> <!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]--> <!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endi
[lookupplayer.php] non-JSON or empty. HTTP 429 | head: <!DOCTYPE html> <!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]--> <!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endi

• Matty Cash | id: 34161548 | team: Aston Villa | pos: Right-Back

• Tyrone Mings | id: 34147598 | team: Aston Villa | pos: Centre-Back

• Matt Targett | id: 34145735 | team: New

In [79]:
import requests, time

BASE = "https://www.thesportsdb.com/api/v1/json/3"

def GET(ep, params=None):
    r = requests.get(f"{BASE}/{ep}", params=params or {}, timeout=20)
    r.raise_for_status()
    return r.json()

TEAM_NAME = "Auckland FC"  # change as needed

# 1) resolve teamId by name
team = (GET("searchteams.php", {"t": TEAM_NAME}).get("teams") or [None])[0]
assert team and team.get("idTeam"), f"Team not found: {TEAM_NAME}"
TEAM_ID = team["idTeam"]

# 2) roster listing (flaky on free tier; add a sanity filter by strTeam)
raw_players = GET("lookup_all_players.php", {"id": TEAM_ID}).get("player") or []
players = [p for p in raw_players if (p.get("strTeam") or "").strip().lower() == TEAM_NAME.lower()]

print(f"Team: {TEAM_NAME} ({TEAM_ID}) | raw={len(raw_players)} | after team filter={len(players)}")
ids = [p["idPlayer"] for p in players if p.get("idPlayer")]
print("Sample ids:", ids[:8])

Team: Auckland FC (149411) | raw=30 | after team filter=0
Sample ids: []


In [80]:
def lookup_player(pid, backoff=0.8, retries=2):
    for i in range(retries+1):
        try:
            j = GET("lookupplayer.php", {"id": pid})
            arr = j.get("players") or []
            if arr:
                return arr[0]
        except Exception:
            pass
        time.sleep(backoff * (i + 1))
    return {}

for pid in ids[:10]:  # limit to avoid 429s; increase carefully
    p = lookup_player(pid)
    print(f"• {p.get('strPlayer')} | id={pid} | team={p.get('strTeam')} | pos={p.get('strPosition')}")
    